In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
np.random.seed(0)

In [24]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };

    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }

        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

<IPython.core.display.Javascript object>

In [62]:
data = pd.read_csv("Cristano_Ronaldo_Final_v1/data.csv")
data = data.drop(data.columns[0],axis=1)
# data.head()

# purged features:
data = data.drop(['shot_id_number'],axis=1)
data = data.drop(['date_of_game'],axis=1)
data = data.drop(['team_id'],axis=1)
# data = data.drop(['type_of_shot'],axis=1)
# data = data.drop(['game_season'], axis=1)
# data = data[data['shot_id_number'].notnull()]

data = pd.get_dummies(data)

# data = data.drop(["type_of_shot_shot - 37","home/away_MANU @ NOK","shot_basics_Right Corner","home/away_MANU vs. NOP","lat/lng_40.324211, -111.674849","type_of_shot_shot - 33",
# "home/away_MANU vs. VAN",
# "type_of_combined_shot_shot - 2",
# "shot_basics_Left Corner",
# "type_of_shot_shot - 34",
# "home/away_MANU vs. CHH",
# "home/away_MANU @ VAN",
# "lat/lng_49.250068, -123.114646",
# "lat/lng_30.055498, -89.960838",
# "lat/lng_35.205878, -80.841194",
# "home/away_MANU @ NOP",
# "home/away_MANU @ CHH",
# "home/away_MANU vs. PHO",
# "range_of_shot_Back Court Shot",
# "area_of_shot_Mid Ground(MG)",
# "game_season_2013-14",
# "home/away_MANU vs. SAN",
# "lat/lng_33.513157, -112.082793",
# "home/away_MANU @ PHO",
# "home/away_MANU @ BKN",
# "lat/lng_40.623199, -73.951223",
# "shot_basics_Mid Ground Line",
# "home/away_MANU @ UTH",
# "home/away_MANU vs. BKN"],axis=1)
# data.head()

In [63]:
#######LOCAL TRAIN-VALIDATION SPLIT########
#Rescale all feature values:
mm_scaler = preprocessing.MinMaxScaler()
std_scaler = preprocessing.StandardScaler()
#global train-test split:
train = data[data['is_goal'].notnull()]
y = train['is_goal']
test = data[data['is_goal'].isnull()]

# local training / validation performance:
train['local_train']=np.random.uniform(0,1,len(train))<=0.85
local_train, local_validation = train[train['local_train']==True], train[train['local_train']==False]
y_local_train = local_train['is_goal']
y_local_validation=local_validation['is_goal']
local_train=local_train.drop(['is_goal'],axis=1)
local_validation =local_validation.drop(['is_goal'],axis=1)

features = local_train.columns[:-1]
# print(features)
local_train = local_train[features]
local_validation = local_validation[features]
print("local train size:", len(local_train))
print("local validation size:", len(local_validation))
print("test size:", len(test))

# preprocessing local-train+validation:
imp= SimpleImputer(missing_values=np.nan,strategy='mean')
imp = imp.fit(local_train)
local_train_modified = imp.transform(local_train)

#scaling
std_scaler.fit(local_train_modified)
StandardScaler(copy=True, with_mean=True, with_std=True)
local_train_modified = std_scaler.transform(local_train_modified)
imp = imp.fit(local_validation)
local_validation_modified = imp.transform(local_validation)
local_validation_modified = std_scaler.transform(local_validation_modified)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


local train size: 20798
local validation size: 3631
test size: 6268


In [64]:
#RF classifier for train-validation perf:
clf = RandomForestClassifier(n_jobs=2, n_estimators=100, random_state=0)
clf.fit(local_train_modified, y_local_train)
p = clf.predict_proba(local_validation_modified)
y_validation_pred = []
for x,y in p:
    y_validation_pred.append(y)
count_match = 0
count_error = 0
deviation = 0.0
assert(len(y_validation_pred)==len(y_local_validation))
validation_gtruth=np.asarray(y_local_validation)
for i in range(len(y_local_validation)):
    deviation += abs(y_validation_pred[i]-validation_gtruth[i])
    if (int(y_validation_pred[i])==int(validation_gtruth[i])):
        count_match+=1
    else:
        count_error+=1
validation_accuracy = count_match/(count_match+count_error)*100.0
print("validation a/c:", validation_accuracy)
print("score:", 1.0/(1.0+deviation/1.0*(count_match+count_error)))
features_imp = pd.DataFrame(clf.feature_importances_, index=features,columns=['importance']).sort_values('importance', ascending=False)
print(features_imp)

validation a/c: 55.27402919305976
score: 1.6837123607036255e-07
                                importance
match_id                          0.056892
match_event_id                    0.056813
remaining_sec.1                   0.053236
remaining_sec                     0.051436
location_y                        0.051244
location_x                        0.049600
remaining_min.1                   0.044280
distance_of_shot.1                0.042888
distance_of_shot                  0.038160
remaining_min                     0.036731
power_of_shot.1                   0.033929
knockout_match.1                  0.026112
power_of_shot                     0.021277
lat/lng_42.982923, -71.446094     0.010382
knockout_match                    0.008876
type_of_combined_shot_shot - 3    0.008781
type_of_shot_shot - 39            0.008776
shot_basics_Goal Area             0.007720
type_of_combined_shot_shot - 1    0.007697
area_of_shot_Center(C)            0.007170
range_of_shot_Less Than 8 ft.    

In [ ]:
#######ACTUAL TRAIN-TEST PERDICTION########
y_train = train['is_goal']
train=train.drop(['is_goal'],axis=1)
test =test.drop(['is_goal'],axis=1)
features = local_train.columns[:-1]
train = train[features]
test = test[features]
print("Train size:", len(train))
print("Test size:", len(test))

# preprocessing local-train+validation:
imp= SimpleImputer(missing_values=np.nan,strategy='mean')
imp = imp.fit(train)
train_modified = imp.transform(train)
imp = imp.fit(test)
test_modified = imp.transform(test)

In [49]:
#NN classifier for train-validation perf:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),learning_rate='constant',learning_rate_init=0.01,solver='sgd',n_iter_no_change=100,max_iter=500,verbose=True)
mlp.fit(local_train_modified,y_local_train)


Iteration 1, loss = 0.69236860
Iteration 2, loss = 0.67354029
Iteration 3, loss = 0.66293003
Iteration 4, loss = 0.65479031
Iteration 5, loss = 0.64879819
Iteration 6, loss = 0.64436252
Iteration 7, loss = 0.64083090
Iteration 8, loss = 0.63807425
Iteration 9, loss = 0.63563544
Iteration 10, loss = 0.63312026
Iteration 11, loss = 0.63130469
Iteration 12, loss = 0.62955053
Iteration 13, loss = 0.62745305
Iteration 14, loss = 0.62611035
Iteration 15, loss = 0.62448870
Iteration 16, loss = 0.62278500
Iteration 17, loss = 0.62134776
Iteration 18, loss = 0.62001036
Iteration 19, loss = 0.61835805
Iteration 20, loss = 0.61714664
Iteration 21, loss = 0.61573032
Iteration 22, loss = 0.61430002
Iteration 23, loss = 0.61272679
Iteration 24, loss = 0.61159315
Iteration 25, loss = 0.61005431
Iteration 26, loss = 0.60862868
Iteration 27, loss = 0.60720879
Iteration 28, loss = 0.60593288
Iteration 29, loss = 0.60453354
Iteration 30, loss = 0.60323390
Iteration 31, loss = 0.60207998
Iteration 32, los

Iteration 255, loss = 0.50000945
Iteration 256, loss = 0.50260851
Iteration 257, loss = 0.49930288
Iteration 258, loss = 0.49978783
Iteration 259, loss = 0.49881483
Iteration 260, loss = 0.49667865
Iteration 261, loss = 0.49675622
Iteration 262, loss = 0.49904449
Iteration 263, loss = 0.49990633
Iteration 264, loss = 0.49923735
Iteration 265, loss = 0.49707122
Iteration 266, loss = 0.49675035
Iteration 267, loss = 0.49813810
Iteration 268, loss = 0.49655665
Iteration 269, loss = 0.49570842
Iteration 270, loss = 0.49597934
Iteration 271, loss = 0.49683165
Iteration 272, loss = 0.49534628
Iteration 273, loss = 0.49748063
Iteration 274, loss = 0.49620063
Iteration 275, loss = 0.49599014
Iteration 276, loss = 0.49593399
Iteration 277, loss = 0.49599993
Iteration 278, loss = 0.49568637
Iteration 279, loss = 0.49466459
Iteration 280, loss = 0.49363093
Iteration 281, loss = 0.49583472
Iteration 282, loss = 0.49617000
Iteration 283, loss = 0.49265267
Iteration 284, loss = 0.49524538
Iteration 

/home/arneish/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(13, 13, 13), learning_rate='constant',
       learning_rate_init=0.01, max_iter=500, momentum=0.9,
       n_iter_no_change=100, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [53]:
#NN predictor:
y_validation_pred = mlp.predict(local_validation_modified)
count_match = 0
count_error = 0
assert(len(y_validation_pred)==len(y_local_validation))
validation_gtruth=np.asarray(y_local_validation)
for i in range(len(y_local_validation)):
    if (int(y_validation_pred[i])==int(validation_gtruth[i])):
        count_match+=1
    else:
        count_error+=1
validation_accuracy = count_match/(count_match+count_error)*100.0
print("validation a/c:", validation_accuracy)

validation a/c: 55.78098943857699


In [174]:
#RF classifier for train-test perf:
clf2 = RandomForestClassifier(n_jobs=2, n_estimators=100, random_state=0)
clf2.fit(train_modified, y_train)
# y_test_pred = clf2.predict(test_modified)
p = clf2.predict_proba(test_modified)
prediction = []
for x,y in p:
    prediction.append(y)
count_match = 0
count_error = 0

# assert(len(y_test_pred)==len(y_local_validation))
# validation_gtruth=np.asarray(y_local_validation)
# for i in range(len(y_local_validation)):
#     if (int(y_validation_pred[i])==int(validation_gtruth[i])):
#         count_match+=1
#     else:
#         count_error+=1
# validation_accuracy = count_match/(count_match+count_error)*100.0
# print("validation a/c:", validation_accuracy)

Train size: 24429
Test size: 6268


In [177]:
#write outputs:
# shot_arr = np.asarray(test['shot_id_number'])
test_rows = data.index[data.is_goal.isnull()]
count = 0
f = open("submissionAP.csv","w+")
print("shot_id_number,is_goal", file=f)
# print("shot_id_number,is_goal", file=f)
for i in range(len(test_rows)):
    print(str(int(test_rows[i]+1))+","+str(prediction[i]), file=f)
    count+=1
print(count)
f.close()

6268


In [124]:
print(validation_gtruth)

[1. 1. 1. ... 0. 0. 0.]


In [142]:
print(len(test))

6268


In [185]:
# list(zip(train[features],clf2.feature_importances_))
features_imp = pd.DataFrame(clf.feature_importances_, index=features,columns=['importance']).sort_values('importance', ascending=False)
print(features_imp)

                                     importance
match_event_id                     4.062614e-02
match_id                           4.043662e-02
distance_of_shot.1                 3.886515e-02
location_y                         3.806000e-02
remaining_sec                      3.752482e-02
team_id                            3.705409e-02
location_x                         3.665255e-02
game_season_1996-97                3.538272e-02
power_of_shot.1                    3.279888e-02
remaining_min                      2.895486e-02
distance_of_shot                   2.892361e-02
knockout_match.1                   2.569584e-02
remaining_sec.1                    1.979802e-02
power_of_shot                      1.720950e-02
shot_basics_Goal Line              8.132400e-03
type_of_combined_shot_shot - 4     7.947864e-03
lat/lng_43.062206, -87.944754      7.935073e-03
area_of_shot_Left Side Center(LC)  6.941969e-03
knockout_match                     6.820156e-03
type_of_combined_shot_shot - 2     6.660

In [11]:
print(local_validation_modified)

[[ 0.10531942 -0.67083244 -0.01123668 ... -0.78672968 -0.30885504
  -0.05429942]
 [ 0.30404697 -0.37358089 -0.01123668 ... -0.78672968 -0.30885504
  -0.05429942]
 [ 0.40683708 -0.94021666  1.71011053 ... -0.78672968 -0.30885504
  -0.05429942]
 ...
 [ 0.06420337 -0.14135312 -0.01123668 ...  1.27108462 -0.30885504
  -0.05429942]
 [ 0.52333254 -0.17850956  6.86167712 ...  1.27108462 -0.30885504
  -0.05429942]
 [ 1.35935879  0.22092221  2.04887568 ... -0.78672968 -0.30885504
  -0.05429942]]


In [150]:
print(test_rows)

Int64Index([    0,     7,    16,    19,    21,    32,    33,    34,    35,
               36,
            ...
            30659, 30664, 30668, 30679, 30680, 30681, 30682, 30686, 30687,
            30693],
           dtype='int64', length=6268)
